# Objective and Motivation

## Objective

## Overview of Wind Turbine Anatomy

## Wind Turbine Aerodynamics

# Code

### Pip Install, Init, and Start Pyspark

In [ ]:
from pyspark.sql import SparkSession
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import col
from pyspark.sql.functions import *
import seaborn as sns

%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from ydata_profiling import ProfileReport

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [3]:
# Get the number of cores
num_cores = sc.defaultParallelism

# Get the number of executors
num_executors = sc.getConf().get("spark.executor.instances")

print(num_cores)
print(num_executors)

8
None


In [4]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Import Data 

In [5]:
df = (spark.read
      .option("header", "true")
      .option("inferSchema", "true")
      .csv("../data/Turbine_Data.csv"))

In [6]:
df.printSchema()

root
 |-- _c0: timestamp (nullable = true)
 |-- ActivePower: double (nullable = true)
 |-- AmbientTemperatue: double (nullable = true)
 |-- BearingShaftTemperature: double (nullable = true)
 |-- Blade1PitchAngle: double (nullable = true)
 |-- Blade2PitchAngle: double (nullable = true)
 |-- Blade3PitchAngle: double (nullable = true)
 |-- ControlBoxTemperature: double (nullable = true)
 |-- GearboxBearingTemperature: double (nullable = true)
 |-- GearboxOilTemperature: double (nullable = true)
 |-- GeneratorRPM: double (nullable = true)
 |-- GeneratorWinding1Temperature: double (nullable = true)
 |-- GeneratorWinding2Temperature: double (nullable = true)
 |-- HubTemperature: double (nullable = true)
 |-- MainBoxTemperature: double (nullable = true)
 |-- NacellePosition: double (nullable = true)
 |-- ReactivePower: double (nullable = true)
 |-- RotorRPM: double (nullable = true)
 |-- TurbineStatus: double (nullable = true)
 |-- WTG: string (nullable = true)
 |-- WindDirection: double (nul

In [7]:
df_renamed = df.withColumnRenamed("_c0", "datetime_stamp")

In [ ]:
df_casted = df_renamed.withColumn("datetime_stamp", col("datetime_stamp").cast("timestamp")).drop(col("_c0"))
df_casted.printSchema()

root
 |-- datetime_stamp: timestamp (nullable = true)
 |-- ActivePower: double (nullable = true)
 |-- AmbientTemperatue: double (nullable = true)
 |-- BearingShaftTemperature: double (nullable = true)
 |-- Blade1PitchAngle: double (nullable = true)
 |-- Blade2PitchAngle: double (nullable = true)
 |-- Blade3PitchAngle: double (nullable = true)
 |-- ControlBoxTemperature: double (nullable = true)
 |-- GearboxBearingTemperature: double (nullable = true)
 |-- GearboxOilTemperature: double (nullable = true)
 |-- GeneratorRPM: double (nullable = true)
 |-- GeneratorWinding1Temperature: double (nullable = true)
 |-- GeneratorWinding2Temperature: double (nullable = true)
 |-- HubTemperature: double (nullable = true)
 |-- MainBoxTemperature: double (nullable = true)
 |-- NacellePosition: double (nullable = true)
 |-- ReactivePower: double (nullable = true)
 |-- RotorRPM: double (nullable = true)
 |-- TurbineStatus: double (nullable = true)
 |-- WTG: string (nullable = true)
 |-- WindDirection: 

In [ ]:
# df_filtered = df_casted.where(
#     to_date(col("datetime_stamp")) =="2018-01-01"
#     )

In [ ]:
pdf = df_casted.toPandas()

In [ ]:
profile = ProfileReport(pdf, title="YData Profiling Report")
profile.to_file('profiling_report.html');

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 22/22 [00:03<00:00,  6.00it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Drop Empty or Single Value Cols

In [29]:
pdf_drop_bad_cols = pdf.drop(columns=['ControlBoxTemperature', 'TurbineStatus', 'WTG'], axis=1)

In [34]:
type(pdf_drop_bad_cols)

pandas.core.frame.DataFrame

In [48]:
corr_matrix = pdf_drop_bad_cols.corr()
corr_matrix


,datetime_stamp,ActivePower,AmbientTemperatue,BearingShaftTemperature,Blade1PitchAngle,Blade2PitchAngle,Blade3PitchAngle,GearboxBearingTemperature,GearboxOilTemperature,GeneratorRPM,GeneratorWinding1Temperature,GeneratorWinding2Temperature,HubTemperature,MainBoxTemperature,NacellePosition,ReactivePower,RotorRPM,WindDirection,WindSpeed
datetime_stamp,1.000000,-0.029146,-0.107832,-0.088521,0.032461,0.030255,0.030255,-0.111076,-0.197239,-0.091271,-0.145035,-0.145350,-0.137042,-0.074034,-0.062090,0.212018,-0.094242,-0.062090,-0.019445
ActivePower,-0.029146,1.000000,-0.065637,0.655400,-0.368997,-0.367978,-0.367978,0.818850,0.821921,0.849604,0.931390,0.932521,0.347363,0.101627,0.028108,0.719092,0.849069,0.028108,0.940390
AmbientTemperatue,-0.107832,-0.065637,1.000000,0.246505,0.085461,0.090904,0.090904,0.017216,0.162511,-0.130123,0.077859,0.078736,0.589729,0.836539,-0.037498,-0.020313,-0.129290,-0.037498,-0.095308
BearingShaftTemperature,-0.088521,0.655400,0.246505,1.000000,-0.475557,-0.466856,-0.466856,0.883344,0.772887,0.640947,0.765115,0.763926,0.809116,0.542104,0.183929,0.594546,0.640584,0.183929,0.596201
Blade1PitchAngle,0.032461,-0.368997,0.085461,-0.475557,1.000000,0.997774,0.997774,-0.599178,-0.557949,-0.757056,-0.376102,-0.372369,-0.198583,0.194911,-0.058300,-0.381478,-0.757147,-0.058300,-0.403092
Blade2PitchAngle,0.030255,-0.367978,0.090904,-0.466856,0.997774,1.000000,1.000000,-0.592103,-0.549792,-0.754325,-0.371057,-0.367451,-0.190296,0.190562,-0.057624,-0.379541,-0.754539,-0.057624,-0.402786
Blade3PitchAngle,0.030255,-0.367978,0.090904,-0.466856,0.997774,1.000000,1.000000,-0.592103,-0.549792,-0.754325,-0.371057,-0.367451,-0.190296,0.190562,-0.057624,-0.379541,-0.754539,-0.057624,-0.402786
GearboxBearingTemperature,-0.111076,0.818850,0.017216,0.883344,-0.599178,-0.592103,-0.592103,1.000000,0.906020,0.850209,0.853024,0.851923,0.596065,0.307694,0.214486,0.751875,0.850795,0.214486,0.800433
GearboxOilTemperature,-0.197239,0.821921,0.162511,0.772887,-0.557949,-0.549792,-0.549792,0.906020,1.000000,0.806972,0.899693,0.898335,0.506648,0.190350,0.277619,0.770088,0.807776,0.277619,0.793140
GeneratorRPM,-0.091271,0.849604,-0.130123,0.640947,-0.757056,-0.754325,-0.754325,0.850209,0.806972,1.000000,0.796434,0.795223,0.303804,-0.052185,0.176964,0.768065,0.999723,0.176964,0.854787


In [ ]:
corr_matrix.abs().unstack()["NacellePosition"]

In [54]:
corr_matrix.abs().unstack().sort_values(ascending=False).drop_duplicates().head(20)

datetime_stamp                datetime_stamp                  1.000000
GeneratorWinding2Temperature  GeneratorWinding1Temperature    0.999959
RotorRPM                      GeneratorRPM                    0.999723
Blade3PitchAngle              Blade1PitchAngle                0.997774
ActivePower                   WindSpeed                       0.940390
GeneratorWinding2Temperature  ActivePower                     0.932521
GeneratorWinding1Temperature  ActivePower                     0.931390
GearboxBearingTemperature     GearboxOilTemperature           0.906020
GeneratorWinding1Temperature  GearboxOilTemperature           0.899693
GeneratorWinding2Temperature  GearboxOilTemperature           0.898335
WindSpeed                     GeneratorWinding2Temperature    0.894834
GeneratorWinding1Temperature  WindSpeed                       0.894777
BearingShaftTemperature       GearboxBearingTemperature       0.883344
RotorRPM                      WindSpeed                       0.855584
WindSp

In [ ]:
### Drop highly correlated features
### pdf[["WindSpeed","ActivePower"]]
### pdf[["RotorRPM", "GeneratorRPM"]]
### pdf[["AmbientTemperatue", "MainBoxTemperature"]]
### pdf[["Blade1PitchAngle","Blade2PitchAngle", "Blade3PitchAngle"]]
### pdf[["NacellePosition","WindDirection"]]


df_drop_corr_cols = df_casted.drop("NacellePosition","RotorRPM", "AmbientTemperatue", "Blade2PitchAngle", "Blade3PitchAngle")


In [ ]:
df_drop_cols = df_drop_corr_cols.drop("")